In [1]:
import pandas as pd
import numpy as np
import math
from geopy import distance

In [2]:
# https://open.canada.ca/data/en/dataset/00a331db-121b-445d-b119-35dbbe3eedd9 -- National Broadband Data
speeds = pd.read_csv("PHH_Speeds_Current-PHH_Vitesses_Actuelles_BC.csv")

# Same source, map data with population information for each hex
phh = pd.read_csv("PHH-BC.csv")
phhWithSpeeds = pd.merge(phh, speeds, on="PHH_ID")

# Filtering out hexes with no population
regionsWithPop = phhWithSpeeds[phhWithSpeeds["Pop2016"] > 0].copy()
regionsWithPop.columns

Index(['PHH_ID', 'Type', 'Pop2016', 'TDwell2016_TLog2016',
       'URDwell2016_RH2016', 'DBUID_Ididu', 'HEXUID_IdUHEX', 'Pruid_Pridu',
       'Latitude', 'Longitude', 'Combined_lt5_1_Combine',
       'Wired_lt5_1_Filaire', 'Wireless_lt5_Sans_fil', 'Combined_5_1_Combine',
       'Wired_5_1_Filaire', 'Wireless_5_1_Sans_fil', 'Combined_10_2_Combine',
       'Wired_10_2_Filaire', 'Wireless_10_2_Sans_fil', 'Combined_25_5_Combine',
       'Wired_25_5_Filaire', 'Wireless_25_5_Sans_fil',
       'Combined_50_10_Combine', 'Wired_50_10_Filaire',
       'Wireless_50_10_Sans_fil', 'Combined_Max_Threshold-Combine_Seuil_Max',
       'Wired_Max_Threshold-Filaire_Seuil_Max',
       'Wireless_Max_Threshold-Sans_fil_Seuil_Max', 'Avail_LTE_Mobile_Dispo'],
      dtype='object')

In [3]:
print(regionsWithPop[['Combined_lt5_1_Combine', 
                      'Combined_10_2_Combine',
                      'Combined_5_1_Combine',
                      'Combined_25_5_Combine']].describe().to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Combined\_lt5\_1\_Combine &  Combined\_10\_2\_Combine &  Combined\_5\_1\_Combine &  Combined\_25\_5\_Combine \\
\midrule
count &           420594.000000 &          420594.000000 &         420594.000000 &          420594.000000 \\
mean  &                0.938097 &               0.889478 &              0.922434 &               0.871981 \\
std   &                0.240979 &               0.313540 &              0.267489 &               0.334111 \\
min   &                0.000000 &               0.000000 &              0.000000 &               0.000000 \\
25\%   &                1.000000 &               1.000000 &              1.000000 &               1.000000 \\
50\%   &                1.000000 &               1.000000 &              1.000000 &               1.000000 \\
75\%   &                1.000000 &               1.000000 &              1.000000 &               1.000000 \\
max   &                1.000000 &               1.000000 &              1.0

In [4]:
school_report = pd.read_csv("combined_school_report.csv")
school_report

,Unnamed: 0,SCHOOL_YEAR,SCHOOL_NUMBER,SCHOOL_NAME_x,DISTRICT_NUMBER,DISTRICT_NAME,PUBLIC_OR_INDEPENDENT,STREET_ADDRESS,PHYSICAL_ADDRESS_CITY,FACILTY_TYPE,...,ENROLMENT,PERCENT_ESL,PERCENT_SPECIAL_NEEDS,PERCENT_FRENCH_IMM,AVERAGE_EXAM_MARK,PERCENT_EXAMS_FAILED,PERCENT_GRAD,PERCENT_DELAYED,OVERALL,SCHOOL_NAME_y.1
0,0,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,...,Gr 12 enrolment: 257,0.4,14.4,6.2,66.9,8.9,99.2,8.2,6.8,Mount Baker
1,1,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,...,Gr 12 enrolment: 257,0.4,14.4,6.2,66.4,10.6,96.8,16.9,6.1,Mount Baker
2,2,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,...,Gr 12 enrolment: 257,0.4,14.4,6.2,66.2,9.6,99.2,5.1,6.6,Mount Baker
3,3,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,...,Gr 12 enrolment: 257,0.4,14.4,6.2,68.0,8.7,97.8,9.7,6.7,Mount Baker
4,4,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,...,Gr 12 enrolment: 257,0.4,14.4,6.2,66.4,8.8,99.6,7.3,6.7,Mount Baker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,1170,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,...,Gr 12 enrolment: 81,0.0,16.6,4.1,66.8,11.1,97.6,9.1,5.8,Nechako Valley
1171,1171,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,...,Gr 12 enrolment: 81,0.0,16.6,4.1,65.6,13.2,96.2,8.3,5.1,Nechako Valley
1172,1172,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,...,Gr 12 enrolment: 81,0.0,16.6,4.1,67.0,12.6,98.0,3.3,6.4,Nechako Valley
1173,1173,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,...,Gr 12 enrolment: 81,0.0,16.6,4.1,71.7,1.6,98.4,7.8,6.9,Nechako Valley


In [5]:
year_2019 = school_report[school_report["YEAR"] == 2019]
print(year_2019[['ENROLMENT',
                'PERCENT_ESL',
                 'PERCENT_SPECIAL_NEEDS',
                 'AVERAGE_EXAM_MARK',
                 'PERCENT_GRAD',
                 'OVERALL']].describe().to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  PERCENT\_ESL &  PERCENT\_SPECIAL\_NEEDS &  AVERAGE\_EXAM\_MARK &  PERCENT\_GRAD &     OVERALL \\
\midrule
count &   235.000000 &             235.000000 &         235.000000 &    235.000000 &  235.000000 \\
mean  &     3.379574 &              11.887234 &          69.148085 &     97.065532 &    6.011915 \\
std   &     4.283132 &               4.780007 &           4.837914 &      2.689002 &    1.761030 \\
min   &     0.000000 &               0.000000 &          54.200000 &     86.700000 &    0.000000 \\
25\%   &     0.150000 &               8.900000 &          66.300000 &     95.300000 &    4.950000 \\
50\%   &     1.900000 &              11.500000 &          68.600000 &     97.700000 &    6.000000 \\
75\%   &     5.100000 &              14.650000 &          71.650000 &     99.100000 &    7.050000 \\
max   &    21.400000 &              29.100000 &          85.400000 &    100.000000 &   10.000000 \\
\bottomrule
\end{tabular}



In [6]:
school_representative = school_report.groupby("SCHOOL_NUMBER").first().reset_index()
regionsWithPop["DISTANCE"] = -1
regionsWithPop["SCHOOL_NUMBER"] = -1
for index, hexagon in regionsWithPop.iterrows():
    hexagon_coord = (hexagon["Latitude"], hexagon["Longitude"])
    closest_school_num = None
    closest_distance = None
    for i, school in school_representative.iterrows():
        dist = distance.distance(hexagon_coord, (school["SCHOOL_LATITUDE"], school["SCHOOL_LONGITUDE"])).km
        if (closest_distance is None or closest_distance > dist):
            closest_distance = dist
            closest_school_num = school["SCHOOL_NUMBER"]
    regionsWithPop.loc[index, "DISTANCE"] = closest_distance
    regionsWithPop.loc[index, "SCHOOL_NUMBER"] = closest_school_num

regionsWithPop.to_csv("pop_regions_with_school.csv")
regionsWithPop.groupby("SCHOOL_NUMBER").size().reset_index(name='counts')

,SCHOOL_NUMBER,counts
0,502001,4296
1,505033,352
2,505034,2180
3,505035,659
4,603008,2635
...,...,...
230,8585026,1821
231,9156002,1515
232,9156007,1048
233,9191023,1679


In [12]:
linked = pd.read_csv("pop_regions_with_school.csv")

# Discard hexes where the closest school is over 25km away -- arbitrary assumption that students aren't traveling that far for school
discard_hex_too_far_away = linked[linked["DISTANCE"] <= 25]
print("Number of hexes discarded: %d" % (linked.size - discard_hex_too_far_away.size))
discard_hex_too_far_away

Number of hexes discarded: 1484192


,Unnamed: 0,PHH_ID,Type,Pop2016,TDwell2016_TLog2016,URDwell2016_RH2016,DBUID_Ididu,HEXUID_IdUHEX,Pruid_Pridu,Latitude,...,Wireless_25_5_Sans_fil,Combined_50_10_Combine,Wired_50_10_Filaire,Wireless_50_10_Sans_fil,Combined_Max_Threshold-Combine_Seuil_Max,Wired_Max_Threshold-Filaire_Seuil_Max,Wireless_Max_Threshold-Sans_fil_Seuil_Max,Avail_LTE_Mobile_Dispo,DISTANCE,SCHOOL_NUMBER
0,12976,4966173,3,3.875229,1.365138,1.188991,59150968013,BC49251232,59,49.257718,...,0,1,1,0,50_10,50_10,5_1,1,0.988530,3939006
1,12977,4957793,4,21.000000,9.000000,7.750000,59150591005,BC49251232,59,49.257987,...,0,1,1,0,50_10,50_10,5_1,1,0.972418,3939006
2,12978,5015696,3,8.061241,4.964545,4.669621,59152437024,BC49231226,59,49.213994,...,0,1,1,0,50_10,50_10,NaN,1,1.532213,4242038
3,12979,5015622,4,27.385336,16.750697,16.385560,59152436001,BC49231226,59,49.213935,...,0,1,1,0,50_10,50_10,NaN,1,1.502037,4242038
4,12980,5115114,3,19.383687,7.698206,7.540294,59170731001,BC48591234,59,48.591087,...,0,1,1,0,50_10,50_10,NaN,1,8.820649,6363022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420583,661345,5500960,2,5.640777,2.990291,2.310680,59390217004,BC51411170,59,51.416553,...,0,0,0,0,5_1,NaN,5_1,0,14.054022,618005
420584,661346,5500961,2,8.048193,3.855422,3.180723,59390216021,BC51331170,59,51.336647,...,0,1,1,0,50_10,50_10,5_1,1,4.481677,618005
420585,661347,5500962,2,3.566667,2.233333,1.466667,59010143099,BC49511155,59,49.530231,...,0,0,0,0,<5_1,NaN,<5_1,1,20.501970,502001
420592,661355,5500970,2,1.158730,0.380952,0.380952,59070148002,BC49231200,59,49.223080,...,1,0,0,0,25_5,NaN,25_5,1,11.142875,5316004
